# Verification


In [49]:
import numpy as np
import qcgpu
from projectq import MainEngine
import projectq.ops as ops
from projectq.backends import Simulator, CircuitDrawer

In [50]:
def qcgpu_amplitudes(n, depth):
    state = qcgpu.State(n)

    h = qcgpu.gate.h()
    x = qcgpu.gate.x()
    t = qcgpu.gate.t()

    for level in range(depth):
        for q in range(n):
    
            state.apply_gate(h, q)
            state.apply_gate(t, q)

            if q != 0:
                state.apply_controlled_gate(x, q, 0)
        
    return state.amplitudes()

In [58]:
def projectq_amplitudes(n, depth):
#     drawing_engine = CircuitDrawer()
#     eng = MainEngine(drawing_engine)
    sim = Simulator(gate_fusion=True)
    eng = MainEngine(backend=sim, engine_list=[])
    qbits = eng.allocate_qureg(n)

    for level in range(depth):
        for q in qbits:
            ops.H | q
            ops.T | q
            if q != qbits[0]:
                ops.CNOT | (q, qbits[0])


    eng.flush()
    amplitudes = sim.cheat()[1]
    for q in qbits:
        ops.Measure | q
   
    return amplitudes
#     return drawing_engine.get_latex()


In [70]:
def verify(n, depth):
    amps_qcgpu = qcgpu_amplitudes(n,depth)
    amps_projectq = np.transpose(np.array([projectq_amplitudes(n,depth)]))
#     print(amps_qcgpu)
#     print(amps_projectq)
    return np.allclose(amps_qcgpu, amps_projectq, rtol=1e-04)
                        
verify(25,15)

True